## <font color='darkblue'>Generate Mocking Data</font>

In [51]:
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# example of defining the discriminator model
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Reshape
from keras.layers import Conv2DTranspose

from keras.utils.vis_utils import plot_model

# example of training the discriminator model on real and random mnist images
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy import vstack
from numpy.random import rand
from numpy.random import randint
from numpy.random import randn
from numpy import asarray

TEST_CASE_LIST = (
  'A', 'B', 'C', 'D', 'E', 'F', 'G')
NUM_OF_TEST_CASE = len(TEST_CASE_LIST)
TEST_CASE_SEQ_LENGTH = 5

In [59]:
def one_hot_encoding(test_case: str):
  try:
    test_case_pos = TEST_CASE_LIST.index(test_case)
  except ValueError as ex:
    raise ex
  encoding = [0] * NUM_OF_TEST_CASE
  encoding[test_case_pos] = 1
  return encoding

def test_case_sequence_2_df(test_case_seq_list: list[str], labels: list[int]):
  test_data = []
  for test_case_seq in test_case_seq_list:
    encoded_test_case_seuqnece = []
    for test_case in test_case_seq:
      encoded_test_case_seuqnece.append(one_hot_encoding(test_case))
      
    test_data.append(encoded_test_case_seuqnece)

  X = np.array(test_data)
  #print(f'data shape={np_data.shape}\n{np_data}')
  # np_data = np.reshape(np_data, (len(test_case_seq_list), TEST_CASE_SEQ_LENGTH * NUM_OF_TEST_CASE))
  #X = pd.DataFrame(np_data)
  y = np.array(labels)
  return X, y

In [56]:
cr_raw_data_list = [
  ('A', 'B', 'C', 'E', 'G'),
  ('B', 'C', 'A', 'F', 'D'),
  ('C', 'A', 'D', 'G', 'C'),
  ('B', 'D', 'C', 'A', 'F'),
  ('C', 'F', 'D', 'A', 'G'),
  ('A', 'B', 'C', 'F', 'G'),
  ('D', 'A', 'G', 'A', 'B'),
  ('D', 'G', 'C', 'A', 'B'),
  ('G', 'A', 'D', 'C', 'F'),
  ('F', 'D', 'C', 'B', 'A'),
]

miss_raw_data_list = [
  ('B', 'C', 'D', 'G', 'A'),
  ('C', 'C', 'D', 'F', 'B'),
  ('A', 'D', 'D', 'F', 'B'),
  ('G', 'G', 'A', 'F', 'C'),
  ('B', 'C', 'F', 'A', 'E'),
  ('A', 'C', 'B', 'E', 'G'),
  ('E', 'A', 'D', 'C', 'G'),
  ('D', 'D', 'B', 'A', 'A'),
  ('F', 'F', 'B', 'C', 'E'),
]

test_case_list = cr_raw_data_list + miss_raw_data_list
labels = [1] * len(cr_raw_data_list) + [0] * len(miss_raw_data_list)

In [60]:
X, y = test_case_sequence_2_df(test_case_list, labels)
X.shape

(19, 5, 7)

In [62]:
X[:1]

array([[[1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1]]])

## <font color='darkblue'>Define and Train the Discriminator Model</font>
<b>The model must take a sample input test case sequence from our dataset as input and output a classification prediction as to whether the sample is real or fake.</b> This is a binary classification problem:
* **Inputs**: Image with one channel and 28×28 pixels in size.
* **Outputs**: Binary classification, likelihood the sample is real (or fake).

In [63]:
# define the standalone discriminator model
def define_discriminator(in_shape=(5, 7, 1)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [64]:
# define model
model = define_discriminator()

# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 4, 64)          640       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 3, 4, 64)          0         
                                                                 
 dropout (Dropout)           (None, 3, 4, 64)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 64)          36928     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 2, 2, 64)          0         
                                                                 
 dropout_1 (Dropout)         (None, 2, 2, 64)          0         
                                                                 
 flatten (Flatten)           (None, 256)               0

2023-07-04 17:56:26.396978: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [65]:
X_train = np.expand_dims(X, axis=-1)

In [66]:
X_train.shape

(19, 5, 7, 1)

In [67]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

In [68]:
# generate n fake samples with class labels
def generate_fake_samples(n_samples):
    # generate uniform random numbers in [0,1]
    X = rand(TEST_CASE_SEQ_LENGTH * NUM_OF_TEST_CASE * n_samples)
    # reshape into a batch of grayscale images
    X = X.reshape((n_samples, TEST_CASE_SEQ_LENGTH, NUM_OF_TEST_CASE, 1))
    # generate 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [69]:
# train the discriminator model
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_iter):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator on real samples
        _, real_acc = model.train_on_batch(X_real, y_real)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(half_batch)
        # update discriminator on fake samples
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        # summarize performance
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [72]:
%%time
# define the discriminator model
model = define_discriminator()

# fit the model
train_discriminator(model, X_train[:10])

2023-07-04 18:17:13.947998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:14.373983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:14.556651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1

>1 real=48% fake=34%
>2 real=62% fake=45%
>3 real=65% fake=59%
>4 real=62% fake=63%
>5 real=56% fake=80%
>6 real=66% fake=95%


2023-07-04 18:17:14.586221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:14.602901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:14.622732: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1

>7 real=67% fake=95%
>8 real=63% fake=98%
>9 real=64% fake=99%
>10 real=68% fake=100%
>11 real=62% fake=100%
>12 real=61% fake=99%


2023-07-04 18:17:14.800150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:14.818822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:14.834039: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1

...

2023-07-04 18:17:17.091140: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:17.109862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:17.125101: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1

>84 real=100% fake=100%
>85 real=100% fake=100%
>86 real=100% fake=100%
>87 real=100% fake=100%
>88 real=99% fake=100%
>89 real=100% fake=100%
>90 real=100% fake=100%


2023-07-04 18:17:17.292516: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:17.308018: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:17.328413: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1

>91 real=100% fake=100%
>92 real=100% fake=100%
>93 real=100% fake=100%
>94 real=100% fake=100%
>95 real=100% fake=100%
>96 real=100% fake=100%
>97 real=100% fake=100%
>98 real=100% fake=100%
>99 real=100% fake=100%
>100 real=100% fake=100%
CPU times: user 5.15 s, sys: 1.25 s, total: 6.39 s
Wall time: 3.78 s


2023-07-04 18:17:17.503257: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:17.515458: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1]
	 [[{{node Placeholder/_1}}]]
2023-07-04 18:17:17.528422: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [128,1

In this case, the discriminator model learns to tell the difference between real and randomly generated test case sequence very quickly, in about 100 iterations. Now that we know how to define and train the discriminator model, we need to look at developing the generator model.

## <font color='darkblue'>How to Define and Use the Generator Model</font> ([back](#sect0))
<b><font size='3ptx'>The generator model is responsible for creating new, fake but plausible test case sequence.</font></b>